In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import thermo
import os
import netCDF4 as nc

In [2]:
path_in = '.'
path_out = './..'
experiment = '001'

ds_in = xr.open_dataset(path_in+'/CP-MIP-2020-02-02.ERA5-LargeFlower.0500m.HYSPLIT.2.0deg.kpt.nc')

# Interpolate the input data onto a time-mean grid, assuming pressure levels do not change much over the run. This seems reasonable:
# ds_in['zf'].where(ds_in['zf']<3500).std(dim='time').plot(y='nlev')

ds_in['zfm'] = ds_in['zf'].mean('time')
ds_in = ds_in.set_coords(['zfm']).swap_dims({'nlev':'zfm'})

ds_in

<xarray.Dataset> Size: 4MB
Dimensions:         (time: 65, zfm: 249, nlevp1: 250, nlevs: 4, nDS: 1)
Coordinates:
  * time            (time) datetime64[ns] 520B 2020-02-01 ... 2020-02-03T16:0...
    nlev            (zfm) float64 2kB 1.0 2.0 3.0 4.0 ... 247.0 248.0 249.0
  * nlevp1          (nlevp1) float64 2kB 1.0 2.0 3.0 4.0 ... 248.0 249.0 250.0
  * nlevs           (nlevs) float64 32B 1.0 2.0 3.0 4.0
  * nDS             (nDS) int64 8B 0
  * zfm             (zfm) float64 2kB 7.14e+04 7.021e+04 ... 30.17 7.807
Data variables: (12/84)
    zf              (time, zfm) float64 129kB 7.14e+04 7.021e+04 ... 30.17 7.542
    zh              (time, nlevp1) float64 130kB ...
    ps              (time) float64 520B ...
    pres            (time, zfm) float64 129kB ...
    presh           (time, nlevp1) float64 130kB ...
    u               (time, zfm) float64 129kB ...
    ...              ...
    DS              (nDS) <U17 68B ...
    timDS           (nDS) datetime64[ns] 8B ...
    latDS           (nDS) float64 8B ...
    lonDS           (nDS) float64 8B ...
    second          (time) float64 520B ...
    date            (time) timedelta64[ns] 520B ...
Attributes:
    campaign:                           CP-MIP
    flight:                             n/a
    date:                               2020-02-01T00:00:00.000000000
    source:                             ERA5
    source_domain:                      NEEDS ADDING
    source_grid:                        grid0.1x0.1
    creator:                            jan.kazil@noaa.gov with https://githu...
    created:                            2024-05-25T14:01:46.134414
    wilting_point:                      0.1715
    field_capacity:                     0.32275
    ALTITUDE_MODIFICATIONS:             Relative to the original LagTraj forc...
    TRAJECTORY_VELOCITY_MODIFICATIONS:  Relative to the original LagTraj forc...

In [3]:
# Pre-process
zf = np.loadtxt('z-grid.txt')

# Verify there is no ice -> indeed
# ds_in['qi'].sel(zfm=slice(7000,0)).plot(y='zfm')

ds_in['exn'] = (ds_in['pres']/1e5)**(thermo.rd/thermo.cp)
ds_in['thl'] = (ds_in['t'] + thermo.rlv/thermo.cp*ds_in['ql']) / ds_in['exn'] # Linear approximation as in DALES, no ice
# ds_in['thl'].sel(zfm=slice(7000,0)).plot(y='zfm')

ds_in['qt'] = ds_in['q'] + ds_in['ql']
# ds_in['qt'].sel(zfm=slice(7000,0)).plot(y='zfm')

ds_in['thls'] = ds_in['open_sst']*(1e5/ds_in['ps'])**(thermo.rd/thermo.cp)

ds_in['rho'] = ds_in['pres']/thermo.rd/ds_in['t'] # Dry approximation
ds_in['w'] = -ds_in['omega']/ds_in['rho']/thermo.grav # w = -\omega/(g\rho)
# ds_in['w'].sel(zfm=slice(7000,0)).plot(y='zfm')

ds_in['qtadv'] = ds_in['qadv'] + ds_in['qladv']
# ds_in['qtadv'].sel(zfm=slice(7000,0)).plot(y='zfm')

ds_in['thladv'] = (ds_in['tadv'] + thermo.rlv/thermo.cp*ds_in['qladv']) / ds_in['exn']
# ds_in['thladv'].sel(zfm=slice(7000,0)).plot(y='zfm')

ds_in['o3vmr'] = ds_in['o3']*28.9644/47.9982 # molar mass of ozone is ~48 g/mol, of dry air it is 28.964.


In [4]:
# prof.inp.001
t0 = '2020-02-01 16:00:00'

ds_prof = ds_in[['thl', 'qt', 'u', 'v']].sel(time=t0).interp(zfm=zf)
tke = 1 - zf/3000; tke[zf>=3000] = 0.
prof = np.stack((zf,
                 ds_prof['thl'].to_numpy(),
                 ds_prof['qt'].to_numpy(),
                 ds_prof['u'].to_numpy(),
                 ds_prof['v'].to_numpy(),
                 tke)).T
profile_out = os.path.join(path_out, 'prof.inp.'+experiment)
np.savetxt(profile_out, prof, fmt='%12.6g',
           header='\n    height         thl          qt            u            v          TKE')

In [5]:
# scalar.inp.001
zero = np.zeros(zf.shape)

scalar = np.stack((zf,zero,zero)).T
scalar_out = os.path.join(path_out, 'scalar.inp.'+experiment)
np.savetxt(scalar_out, scalar, fmt='%12.6g',
           header='#\nheight  nr  qr', comments='')

In [13]:
# ls_flux.inp.001

tf = '2020-02-02 19:00:00' # 1 hour after simulation ends
ds_sur = ds_in[['thls', 'ps']].sel(time=slice(t0,tf))
ds_for = ds_in[['ug', 'vg', 'w', 'qtadv', 'thladv']].sel(time=slice(t0,tf)).interp(zfm=zf)
time_s = (ds_sur['time'] - ds_sur['time'][0]).dt.total_seconds().to_numpy()

ls_flux_out = os.path.join(path_out, 'ls_flux.inp.'+experiment)

# First write surface forcing
zerot = np.zeros(ds_sur['time'].size)
surf = np.stack((time_s,
                 zerot,
                 zerot,
                 ds_sur['thls'].to_numpy(),
                 zerot,
                 ds_sur['ps'].to_numpy()
                 )).T
f = open(ls_flux_out, 'w')
f.close()
np.savetxt(ls_flux_out, surf, fmt='%+10.9e', comments='',
           header='\n     time           wthl_s           wqt_s            T_s            qt_s             p_s      \n      (s)          (K m s-1)    (kg kg-1 m s-1)       (K)          (kg kg-1)         (Pa)      ')

# Append time instances which interpolate the vertical forcing
with open(ls_flux_out, 'a') as f:
    f.write('\n       z (m)            u_g (m s-1)         v_g (m s-1)        w_ls (m s-1)     dqtdx (kg kg-1 m-1)  dqtdy (kg kg m-1)  dqtdt (kg kg-1 s-1)   dthldt (K s-1)       dudt (m s-2)        dvdt (m s-2)    \n')
    for i in range(len(ds_for['time'])):
        ds_for_i = ds_for.isel(time=i)
        ls_profs = np.stack((zf,
                             ds_for_i['ug'].to_numpy(),
                             ds_for_i['vg'].to_numpy(),
                             ds_for_i['w'].to_numpy(),
                             zero,
                             zero,
                             ds_for_i['qtadv'].to_numpy(),
                             ds_for_i['thladv'].to_numpy(),
                             zero,
                             zero
                           )).T
        f.write('\n# {:14.9e}\n'.format(time_s[i]))
        np.savetxt(f, ls_profs, fmt='%+10.10e')

In [7]:
# lscale.inp.001

# All zeros, but still needed somehow
lscale = np.stack((zf,zero,zero,zero,zero,zero,zero,zero)).T
lscale_out = os.path.join(path_out, 'lscale.inp.'+experiment)
np.savetxt(lscale_out, lscale, fmt='%12.6g',
           header='\n    height           ug           vg         wfls      dqtdxls      dqtdyls      dqtdtls      dthlrad')

In [112]:
# backrad.001.nc
ds_rad = ds_in[['pres','t','q','o3vmr']].mean('time').isel(zfm=slice(None, None, -1))

backrad_out = os.path.join(path_out,'backrad.inp.'+experiment+'.nc')

nc_file = nc.Dataset(backrad_out, 'w')
nc_file.title = 'Background radiation input for cp-mip simulations, from ERA5'

dims = nc_file.createDimension('lev', ds_rad['pres'].size)

p_var = nc_file.createVariable('lev', 'f4', ('lev'))
T_var = nc_file.createVariable('T',   'f4', ('lev'))
q_var = nc_file.createVariable('q',   'f4', ('lev'))
o_var = nc_file.createVariable('o3',  'f4', ('lev'))

p_var.units = 'Pa'
T_var.units = 'K'
q_var.units = 'kg/kg'
o_var.units = '-'

p_var[:] = ds_rad['pres'].to_numpy()
T_var[:] = ds_rad['t'].to_numpy()
q_var[:] = ds_rad['q'].to_numpy()
o_var[:] = ds_rad['o3vmr'].to_numpy()

nc_file.close()

In [98]:
# cu/cv - take to be the mean translation velocity of the trajectory
print('cu:', ds_in['u_traj'].mean().values)
print('cv:', ds_in['v_traj'].mean().values)


cu: -7.024928302512297
cv: -1.140937589836673
